## DATA Preprocess

### Coordinate transform

In [4]:
import numpy as np
import cvxpy as cp

T_z_90 = np.array([[0,-1,0,0],[1,0,0,0],[0,0,1,0],[ 0,0,0,1]])
T_z_min90 = T_z_90.T
R_z_90 = T_z_90[:3, :3]


def coordinate_transform(P0, P1, Pose0, Pose1):
    def Transformation_matrix(Pose0, Pose1):
        # T1 = T0 @ T
        T_matrix = np.linalg.inv(Pose0) @ Pose1 
        # x-axis oriented switched to y-axis oriented
        T_matrix = T_z_90 @ T_matrix @ T_z_min90
        # get transforamtion matrix
        T_matrix = np.linalg.inv(T_matrix)
        return T_matrix
    P0 = (T_z_90 @ P0)[:3]
    P1 = (T_z_90 @ P1)[:3]
    T_matrix = Transformation_matrix(Pose0, Pose1)
    return P0, P1, T_matrix


### Testing example

In [5]:

{'Pose': [[0.9947831471329273, 0.10201220603589002, 0.0, -0.13595595336837749], [-0.10201220603589002, 0.9947831471329273, -0.0, -1.9524370759126866], [-0.0, 0.0, 1.0, -0.3570108264917502], [0.0, 0.0, 0.0, 1.0]], 
 'theta_Rho': [[-0.8427304625511169, 2.9157803058624268]], 
 's_p': [[1.92564058303833, 2.1601450443267822, 0.3570108413696289]], 'w_p': [[2.0, 0.0, 0.0]]}
{'Pose': [[0.8676190607260436, -0.49722948973774467, 0.0, 0.7461405519015483], [0.49722948973774467, 0.8676190607260436, 0.0, -0.5554558458716767], [0.0, 0.0, 1.0, -0.37813731018453933], [0.0, 0.0, 0.0, 1.0]], 
 'theta_Rho': [[0.10338770598173141, 1.4225620031356812]], 
 's_p': [[1.3640613555908203, -0.1415318101644516, 0.3781373202800751]], 'w_p': [[2.0, 0.0, 0.0]]}

theta_Rho = np.array([-0.8427304625511169, 2.9157803058624268]) 
theta_Rho_prime = np.array([0.10338770598173141, 1.4225620031356812]) 
Pose0 =  np.array([[0.9947831471329273, 0.10201220603589002, 0.0, -0.13595595336837749], [-0.10201220603589002, 0.9947831471329273, -0.0, -1.9524370759126866], [-0.0, 0.0, 1.0, -0.3570108264917502], [0.0, 0.0, 0.0, 1.0]])
Pose1 =  np.array([[0.8676190607260436, -0.49722948973774467, 0.0, 0.7461405519015483], [0.49722948973774467, 0.8676190607260436, 0.0, -0.5554558458716767], [0.0, 0.0, 1.0, -0.37813731018453933], [0.0, 0.0, 0.0, 1.0]])
P0 = np.array([1.92564058303833, 2.1601450443267822, 0.3570108413696289, 1])
P1 = np.array([1.3640613555908203, -0.1415318101644516, 0.3781373202800751, 1])

P0, P1, T_matrix = coordinate_transform(P0, P1, Pose0, Pose1)

## Calculate Determinant


In [6]:
def compute_D(T_matrix, theta, theta_prime):
    
    R = T_matrix[:3, :3]
    t = T_matrix[:3, 3]
    
    """
    Compute the determinant D(A0; R, t) for given parameters.
    """
    def skew_symmetric_matrix(t):
        """
        Create a skew-symmetric matrix for a vector t.
        """
        return np.array([
            [0, -t[2], t[1]],
            [t[2], 0, -t[0]],
            [-t[1], t[0], 0]
        ])
    ux = np.array([1, 0, 0])
    uy = np.array([0, 1, 0])
    
    r1 = R[0, :]
    r2 = R[1, :]
        
    t_cross = skew_symmetric_matrix(t)
    
    determinant = - (r1 - np.tan(theta) * r2).T @ t_cross @ (ux - np.tan(theta_prime) * uy)
    
    return determinant

determinant = compute_D(T_matrix, theta_Rho[0], theta_Rho_prime[0])
print("determiant: ", determinant)
##########################################################################


determiant:  -0.03191912774507307


## ANRS

In [7]:

def ANRS(T_matrix, theta_Rho, theta_Rho_prime):

    # 将线性方程组写成矩阵形式 A @ P = B

    R_matrix = T_matrix[:3, :3]
    r1 = R_matrix[0, :]
    r2 = R_matrix[1, :]
    t = T_matrix[:3, 3]


    theta = theta_Rho[0]
    theta_prime = theta_Rho_prime[0]
    R = theta_Rho[1]  # example value for R
    R_prime = theta_Rho_prime[1] # example value for R'
    
    a1 = np.array([-1, np.tan(theta), 0])
    b1 = 0 
    a2 = np.tan(theta_prime) * r2 - r1
    b2 = t[0] - np.tan(theta_prime) * t[1]
    a3 = t.T @ R_matrix
    b3 = (R_prime**2 - R**2 - np.linalg.norm(t)**2) / 2

    A = np.vstack([a1, a2, a3])
    b = np.array([b1, b2, b3])
    ## Testing 
    wanted_result_P = P0
    # print("A @ P = b:\n", (A @ wanted_result_P),"\n should equals to \n", b, "\n" ) # P0 is what we want to get
    
    
    # ANRS
    P_o = np.linalg.inv(A) @ b
    norm_P_o = np.linalg.norm(P_o)
    P_hat_o = P_o / norm_P_o

    A_tilde_prime = np.vstack([A, P_hat_o.T])
    b_tilde_prime = np.append(b, R)

    # 計算 \(\tilde{P}_o'\)
    P_tilde_prime = np.linalg.inv(A_tilde_prime.T @ A_tilde_prime) @ A_tilde_prime.T @ b_tilde_prime      
    
    return P_tilde_prime

ANRS_res = ANRS(T_matrix, theta_Rho, theta_Rho_prime)
print("Calculated P:", ANRS_res)
print("The Ground truth value: ", P0)


Calculated P: [-2.16014499  1.92564049  0.35701138]
The Ground truth value:  [-2.16014504  1.92564058  0.35701084]


## GTRS

In [8]:
import numpy as np
from scipy.linalg import eig

def GTRS(T_matrix, theta_Rho, theta_Rho_prime):

    # Extract components from T_matrix
    R_matrix = T_matrix[:3, :3]
    t = T_matrix[:3, 3]

    # Extract theta, theta_prime, R, R_prime from inputs
    theta = theta_Rho[0]
    theta_prime = theta_Rho_prime[0]
    R = theta_Rho[1]
    R_prime = theta_Rho_prime[1]

    # Define linear constraints in matrix form A @ P = B
    r1 = R_matrix[0, :]
    r2 = R_matrix[1, :]

    a1 = np.array([-1, np.tan(theta), 0, 0])
    b1 = 0 
    a2 = np.tan(theta_prime) * r2 - r1
    a2 = np.hstack([a2, 0])
    b2 = t[0] - np.tan(theta_prime) * t[1]
    a3 = t.T @ R_matrix
    a3 = np.hstack([a3, 0])
    b3 = (R_prime**2 - R**2 - np.linalg.norm(t)**2) / 2
    a4 = np.array([0, 0, 0, 1])
    b4 = R**2

    # 将线性方程组写成矩阵形式 A @ P = B
    A = np.vstack([a1, a2, a3, a4])
    b = np.array([b1, b2, b3, b4])

    ATA_be = A.T @ A
    ATb_be = A.T @ b
    D = np.diag([1,1,1,0])
    g = np.zeros(4)
    g[3] = -0.5

    # eig_lambda = eig(D, ATA_be, right=False)
    eig_lambda = np.real(eig(D, ATA_be, right=False))

    lambda_min = -1 / np.max(eig_lambda)

    # 找二分法的初始左端点
    lambda_l = lambda_min + 2
    y_l = np.linalg.solve(ATA_be + lambda_l * D, ATb_be - lambda_l * g)
    while (y_l.T @ D @ y_l + 2 * g.T @ y_l) < 0:
        lambda_l = (lambda_min + lambda_l) / 2
        y_l = np.linalg.solve(ATA_be + lambda_l * D, ATb_be - lambda_l * g)

    # 找二分法的初始右端点
    lambda_u = lambda_min + 2
    y_u = np.linalg.solve(ATA_be + lambda_u * D, ATb_be - lambda_u * g)
    while (y_u.T @ D @ y_u + 2 * g.T @ y_u) > 0:
        lambda_u = lambda_u + (lambda_u - lambda_min) * 2
        y_u = np.linalg.solve(ATA_be + lambda_u * D, ATb_be - lambda_u * g)

    # 二分法找最优lambda
    while (lambda_u - lambda_l) > 1e-15:
        lambda_temp = (lambda_u + lambda_l) / 2
        y_temp = np.linalg.solve(ATA_be + lambda_temp * D, ATb_be - lambda_temp * g)
        if (y_temp.T @ D @ y_temp + 2 * g.T @ y_temp) > 0:
            lambda_l = lambda_temp
        else:
            lambda_u = lambda_temp

    y = np.linalg.solve(ATA_be + lambda_u * D, ATb_be - lambda_u * g)
    pos = y[:3]
    
    # distance_constraint = y[3] - np.linalg.norm(pos)**2
    # print('Distance constraint value:', distance_constraint)
    
    return pos


# 调用CTOA_GTRS函数
GTRS_res = GTRS(T_matrix, theta_Rho, theta_Rho_prime)

# 显示结果
print('Calculated result:', GTRS_res)
print('Wanted result:', P0)

Calculated result: [-2.16014499  1.92564049  0.35701137]
Wanted result: [-2.16014504  1.92564058  0.35701084]


### Problematic GTRS

In [9]:
# import numpy as np
# import cvxpy as cp

# def GTRS_zeng(T_matrix, theta_Rho, theta_Rho_prime):
#     # Extract components from T_matrix
#     R_matrix = T_matrix[:3, :3]
#     t = T_matrix[:3, 3]

#     # Extract theta, theta_prime, R, R_prime from inputs
#     theta = theta_Rho[0][0]
#     theta_prime = theta_Rho_prime[0][0]
#     R = theta_Rho[0][1]
#     R_prime = theta_Rho_prime[0][1]

#     # Define linear constraints in matrix form A @ P = B
#     r1 = R_matrix[0, :]
#     r2 = R_matrix[1, :]

#     a1 = np.array([-1, np.tan(theta), 0, 0])
#     b1 = 0 
#     a2 = np.tan(theta_prime) * r2 - r1
#     a2 = np.hstack([a2, 0])
#     b2 = t[0] - np.tan(theta_prime) * t[1]
#     a3 = t.T @ R_matrix
#     a3 = np.hstack([a3, 0])
#     b3 = (R_prime**2 - R**2 - np.linalg.norm(t)**2) / 2
#     a4 = np.array([0, 0, 0, 1])
#     b4 = R**2

#     # 将线性方程组写成矩阵形式 A @ P = B
#     A1 = np.vstack([a1, a2, a3, a4])
#     b1 = np.array([b1, b2, b3, b4])

#     D = np.diag([1, 1, 1, 0])
#     g = np.array([0, 0, 0, -0.5])

#     ATA = A1.T @ A1
#     ATb = (A1.T @ b1).reshape(-1, 1)

#     # Define variables
#     t_var = cp.Variable()
#     v = cp.Variable()

#     # Define the problem
#     semi_positive_mat = cp.bmat([[ATA + v * D, ATb - v * g.reshape(-1, 1)], [ATb.T - v * g.reshape(1, -1), cp.reshape(t_var, (1, 1))]])
#     semi_positive_mat_val = semi_positive_mat
#     constraints = [semi_positive_mat>>0]
#     # print(semi_positive_mat_val)
#     objective = cp.Minimize(t_var)

#     prob = cp.Problem(objective, constraints)
#     prob.solve(solver=cp.SCS, verbose=False)

#     # Calculate the result
#     y = np.linalg.inv(ATA + v.value * D) @ (ATb - v.value * g.reshape(-1, 1))
#     x = y[:3].flatten()
    
#     return x


# # 調用函數並打印結果
# result = GTRS_zeng(T_matrix, theta_Rho, theta_Rho_prime)
# print(result)


## Performance Comparation

In [10]:
print(np.sum(ANRS_res-P0)**2)
print(np.sum(GTRS_res-P0)**2)

2.5763303050932375e-13
2.4843507890432953e-13


In [11]:
def read_data2(file_path):
    data_sets = []
    with open(file_path, 'r') as file:
        for line in file:
            if line.strip():  # Skip empty lines
                # Parse the line to extract data
                parts = line.split('; ')
                Pose = eval(parts[0].split('Pose: ')[1])
                theta_Rho = eval(parts[1].split('theta_Rho: ')[1])[0]
                P = eval(parts[2].split('s_p: ')[1])[0]
                data_sets.append([Pose, theta_Rho, P])
    return data_sets

file_path = 'record.txt'
data_sets = read_data2(file_path)
times = 0
for i in range(len(data_sets)):
    for j in range(i + 1, len(data_sets)):
        [Pose0, theta_Rho0, P0] = data_sets[i]
        [Pose1, theta_Rho1, P1] = data_sets[j]
        P0 = np.hstack([P0, 1])
        P1 = np.hstack([P1, 1])
        P0, P1, T_matrix = coordinate_transform(P0, P1, Pose0, Pose1)        
        determinant = compute_D(T_matrix, theta_Rho0[0], theta_Rho1[0])
        ANRS_res = ANRS(T_matrix, theta_Rho0, theta_Rho1)
        GTRS_res = GTRS(T_matrix, theta_Rho0, theta_Rho1)
        
        print("Distance: ", np.sqrt( np.sum((P0-P1)**2)))
        print("determinant: ", j-i, determinant)
        print("T_matrix:\n", T_matrix)
        print(np.sum(ANRS_res-P0)**2)
        print(np.sum(GTRS_res-P0)**2)
        print()
        times += 1
        if times > 20:
            break


Distance:  0.23027711287573954
determinant:  1 0.0008104671265972171
T_matrix:
 [[ 0.99990946  0.0124861  -0.00501686 -0.18368821]
 [-0.01266722  0.99920278 -0.03785961 -0.06077924]
 [ 0.00454014  0.03791973  0.99927047  0.02104766]
 [ 0.          0.          0.          1.        ]]
4.345320807235472e-10
4.3384048751060836e-10

Distance:  0.4007820610673674
determinant:  2 0.0021253403217927997
T_matrix:
 [[ 0.99934816  0.03132265 -0.01794866 -0.33426559]
 [-0.03254454  0.99684447 -0.07240137 -0.14602971]
 [ 0.01562422  0.07293831  0.99721406  0.00459399]
 [ 0.          0.          0.          1.        ]]
7.998122058506846e-12
8.061846424875438e-12

Distance:  0.503310069382022
determinant:  3 0.0062861836583787516
T_matrix:
 [[ 0.99775532  0.05509545 -0.03806332 -0.42904985]
 [-0.05873437  0.99302488 -0.10223437 -0.24537478]
 [ 0.03216517  0.10424051  0.99403185 -0.05455015]
 [ 0.          0.          0.          1.        ]]
4.1541986540406305e-14
5.79639330504072e-14

Distance:  0

In [12]:
for i in range(len(data_sets)-1):
    [Pose0, theta_Rho0, P0] = data_sets[i]
    [Pose1, theta_Rho1, P1] = data_sets[i+1]
    P0 = np.hstack([P0, 1])
    P1 = np.hstack([P1, 1])
    P0, P1, T_matrix = coordinate_transform(P0, P1, Pose0, Pose1)        
    determinant = compute_D(T_matrix, theta_Rho0[0], theta_Rho1[0])
    ANRS_res = ANRS(T_matrix, theta_Rho0, theta_Rho1)
    GTRS_res = GTRS(T_matrix, theta_Rho0, theta_Rho1)
    
    print("Distance: ", np.sqrt( np.sum((P0-P1)**2)))
    print("determinant: ", j-i, determinant)
    print("T_matrix:\n", T_matrix)
    print(np.sum(ANRS_res-P0)**2)
    print(np.sum(GTRS_res-P0)**2)
    print()


Distance:  0.23027711287573954
determinant:  58 0.0008104671265972171
T_matrix:
 [[ 0.99990946  0.0124861  -0.00501686 -0.18368821]
 [-0.01266722  0.99920278 -0.03785961 -0.06077924]
 [ 0.00454014  0.03791973  0.99927047  0.02104766]
 [ 0.          0.          0.          1.        ]]
4.345320807235472e-10
4.3384048751060836e-10

Distance:  0.17445988255701556
determinant:  57 0.0009101851782894301
T_matrix:
 [[ 0.99973882  0.01931824 -0.01221064 -0.1491942 ]
 [-0.01973167  0.9992031  -0.03469623 -0.08819311]
 [ 0.01153064  0.03492811  0.99932331 -0.01219848]
 [ 0.          0.          0.          1.        ]]
3.176765334900259e-12
3.420710647459386e-12

Distance:  0.12697369153244875
determinant:  56 0.002910019826743649
T_matrix:
 [[ 0.99951386  0.02520595 -0.01834955 -0.09118164]
 [-0.02575694  0.99920475 -0.03043767 -0.10793103]
 [ 0.01756775  0.0308955   0.99936822 -0.04875729]
 [ 0.          0.          0.          1.        ]]
7.382011590714407e-12
7.390878884470304e-12

Distanc